In [ ]:
# Then calculae the average word frequency for commit title
    #1) average WF for each comment
    #2) average WF for each project    
# Then calculae the average word frequency for commit title && Body
    #1) average WF for each comment
    #2) average WF for each project  
import re
import json
import string
import numpy as np
import os
frequency_corpus = 'path_to_the_sorted_corpus'
words_not_in_corpus = 'path/file_of_the_words_not_in_corpus_with_their_transformations'
path = "path_to_the_titles_or_titlesAndBodies"

# read the corpus words and frequency ranks into a dict
corpus_dict = {}
fo = open(frequency_corpus,"r",encoding="utf-8") 
for line in fo.readlines():
    line = line.strip()
    if ':' in line:
        k = line.split(':')[0].strip()
        v = line.split(':')[2].strip()
        corpus_dict[k] = v

# read words not in corpus and it's transformation into a dict
tranform_dict = {}
f3 = open(words_not_in_corpus,"r",encoding="utf-8")     
for line in f3.readlines():
    line = line.strip()
    if ':' in line:
        k = line.split(':')[0].strip()
        v = line.split(':')[1].strip()
        tranform_dict[k] = v
    else:
        tranform_dict[line.strip()] = '--'

# For excluding the name extities
p = '[(]?[#][0-9]+'
p1 = '[A-Z]+([_][A-Z]+)+'
p2 = '[a-z][a-z1-9]+([A-Z][a-z1-9]+)+'
p3 = '[A-Z][a-z1-9]+([A-Z][a-z1-9]+)+' 
p4 = '[a-z]([A-Z][a-z]+)+' 
p5 = '[a-z1-9]+([_][a-z1-9]+)+' 
p6 = '[_][_]*(.+)'  
p7 = '[a-z][_]([A-Z]*[a-z1-9]+)+' 
p8 = '[a-z][_](.+)'
p9 = '[_]([a-z]+[A-Z][a-z]+)+'

# read comment titles
files= os.listdir(path)
for file in files:
    pro_id = file.split('.txt')[0]
    pro_words_list = [] 
    if not os.path.isdir(file):
        f2 = open(path + "/" + file,"r",encoding="utf-8")
        line = f2.readline()
        while line:
            each_comment_word_list = line.split()
            pro_words_list.append(each_comment_word_list) 
            line = f2.readline()
            
    # 1. clean project word list 
    pro_word_list_clean = []
    for idx,cur_comm_word_list in enumerate(pro_words_list):        
        each_comm_w_list_clean = []           
        # comment of current project
        for index, val in enumerate(cur_comm_word_list):            
            v = re.match(p,val,flags=0)
            v1 = re.match(p1,val,flags=0)
            v2 = re.match(p2,val,flags=0)
            v3 = re.match(p3,val,flags=0)
            v4 = re.match(p4,val,flags=0)
            v5 = re.match(p5,val,flags=0)
            v6 = re.match(p6,val,flags=0)
            v7 = re.match(p7,val,flags=0)
            v8 = re.match(p8,val,flags=0)
            v9 = re.match(p9,val,flags=0)                        
            # exluding name entities
            if "()" not in val:
                if v == None and v1 == None and v2 == None and v3 == None and v4 == None and v5 == None:
                    if v6 == None and v7 == None and v8 == None and v9 == None:
                        val = val.strip(string.punctuation)
                        if '.' not in val and '(' not in val and ':' not in val and '→' not in val:
                            if len(val) > 0:
                                each_comm_w_list_clean.append(val.lower())                                
        pro_word_list_clean.append(each_comm_w_list_clean)   

    # 2. check frequency rank
    record_all_avg_commit_w_freq = []
    for cur_comm in pro_word_list_clean:  
        # record word frequency ranks for each commit
        record_all_w_rank_of_a_commit = []         
        for cur_word in cur_comm:
            check_w_list = []           
            # cur_word in words_not_in_corpus list, check tranformation
            if cur_word in tranform_dict.keys():                
                if tranform_dict[cur_word] != '--':
                    # multiple words
                    # handle missing space case: quickfix -> quick fix, 
                    # tranform_dict[cur_word] would be more than one word
                    if ' ' in tranform_dict[cur_word]:
                        check_w_list = tranform_dict[cur_word].split(' ')
                        for d in check_w_list:
                            if d in corpus_dict.keys():
                                record_all_w_rank_of_a_commit.append(corpus_dict[d])

                    # single word   
                    elif tranform_dict[cur_word] in corpus_dict.keys():
                        record_all_w_rank_of_a_commit.append(corpus_dict[tranform_dict[cur_word]])
                        
            elif cur_word in corpus_dict.keys():
                record_all_w_rank_of_a_commit.append(corpus_dict[cur_word])
        record_all_w_rank_of_a_commit = np.array(record_all_w_rank_of_a_commit).astype(np.float)

        # average word freqency for each commit
        cur_commit_avg_w_fre = np.mean(record_all_w_rank_of_a_commit)
        record_all_avg_commit_w_freq.append(cur_commit_avg_w_fre)
        
    cur_project_avg_w_fre = np.nanmean(record_all_avg_commit_w_freq)
    print(pro_id,cur_project_avg_w_fre)
fo.close()
f3.close()
f2.close()